In [1]:
import os
import glob
import pandas as pd
from collections import defaultdict

In [2]:
data_path = '../data/'

In [24]:
csv_paths = glob.glob(data_path + '*loss*.csv')
filenames = [os.path.splitext(os.path.basename(path))[0] for path in csv_paths]
scales = set([s[s.rfind('_') + 1:] for s in filenames])
print(f'Found following scale in data: {scales}')

Found following scale in data: {'240', '120', '1920', '960', '480'}


In [76]:
data_by_scale = {}

In [77]:
for scale in scales:
    all_files = glob.glob(data_path + '*' + scale + '.csv')
    loss_files = [path for path in all_files if 'loss' in path]
    df_all_years = []
    for path in loss_files:
        df = pd.read_csv(path)
        df = df.drop(labels='.geo', axis=1)
        year_idx_start = path.rfind('loss_') + len('loss_')
        year = path[year_idx_start: year_idx_start + 4] # 4 = number of digits in "year"
        df.columns = ['species', f'loss_{year}']
        df_all_years.append(df)
        
    df = pd.concat(df_all_years, axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    df = df.reindex(sorted(df.columns), axis=1)
    
    treecover_path = (set(all_files) - set(loss_files)).pop()
    treecover_df = pd.read_csv(treecover_path)
    
    df.insert(0, 'treecover_2000', treecover_df['treecover2000'])
    df = df.set_index('species')
    df.to_csv(f'large_sample_{scale}.csv')
    data_by_scale[scale] = df

240
120
1920
960
480


In [80]:
data_by_scale.keys()

dict_keys(['240', '120', '1920', '960', '480'])

In [82]:
data_by_scale['240'] - data_by_scale['120']

,treecover_2000,loss_2001,loss_2002,loss_2003,loss_2004,loss_2005,loss_2006,loss_2007,loss_2008,loss_2009,loss_2010,loss_2011,loss_2012,loss_2013,loss_2014,loss_2015,loss_2016,loss_2017,loss_2018
species,,,,,,,,,,,,,,,,,,,
Accipiter_gentilis1,3.081416e+10,-3.894666e+09,-4.154392e+09,-3.208994e+09,-5.902776e+09,-5.304381e+09,-5.135093e+09,-5.204482e+09,-5.470225e+09,-4.589509e+09,-5.205236e+09,-4.714932e+09,-5.791310e+09,-3.343072e+09,-4.737109e+09,-3.696669e+09,-5.558076e+09,-5.345967e+09,-5.938272e+09
Accipiter_gentilis3,2.252975e+09,-3.094970e+08,-3.380552e+08,-3.828822e+08,-5.588498e+08,-5.514719e+08,-6.705045e+08,-7.293358e+08,-6.658525e+08,-6.663633e+08,-7.226150e+08,-6.634767e+08,-6.263176e+08,-6.024358e+08,-8.189160e+08,-8.465306e+08,-1.041556e+09,-1.057606e+09,-9.798927e+08
Accipiter_nisus1,6.835340e+09,-1.121661e+09,-1.220542e+09,-9.795945e+08,-1.813337e+09,-1.911505e+09,-1.760626e+09,-1.964967e+09,-2.032479e+09,-1.738318e+09,-2.094007e+09,-1.889376e+09,-1.899685e+09,-1.382614e+09,-1.747000e+09,-1.608154e+09,-2.455625e+09,-2.435196e+09,-2.650529e+09
Accipiter_nisus2,1.425259e+10,-1.403762e+09,-1.481062e+09,-1.209991e+09,-2.123668e+09,-1.700746e+09,-1.771216e+09,-1.614465e+09,-1.985200e+09,-1.673836e+09,-1.809051e+09,-1.566368e+09,-2.368289e+09,-1.168687e+09,-1.606643e+09,-1.316637e+09,-1.576004e+09,-1.677780e+09,-1.798117e+09
Accipiter_nisus3,6.806870e+09,-5.452334e+08,-6.391306e+08,-6.689588e+08,-1.009273e+09,-9.226073e+08,-1.218749e+09,-1.538760e+09,-1.563282e+09,-1.623188e+09,-1.665209e+09,-1.551358e+09,-1.578333e+09,-1.488032e+09,-1.913186e+09,-2.032697e+09,-2.678703e+09,-2.647963e+09,-2.331240e+09
Bucephala_clangula2,2.205822e+10,-2.483933e+09,-2.664025e+09,-2.094126e+09,-3.723475e+09,-3.355669e+09,-3.330104e+09,-3.020901e+09,-3.406139e+09,-2.934945e+09,-3.207666e+09,-3.019816e+09,-3.918624e+09,-1.982650e+09,-3.110088e+09,-2.197239e+09,-3.270084e+09,-3.125003e+09,-3.379657e+09
Bucephala_clangula3,1.076433e+10,-2.320863e+09,-2.258560e+09,-1.754296e+09,-3.163163e+09,-2.828198e+09,-2.895486e+09,-3.102009e+09,-3.158959e+09,-2.566050e+09,-2.757372e+09,-2.639856e+09,-2.823058e+09,-2.016151e+09,-2.388258e+09,-2.197721e+09,-2.945331e+09,-3.016893e+09,-2.988548e+09
Coccyzus_americanus1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Coccyzus_americanus2,3.918654e+09,-1.600217e+09,-1.473286e+09,-1.176943e+09,-1.846411e+09,-1.765817e+09,-1.750545e+09,-1.787904e+09,-1.715784e+09,-1.492609e+09,-1.548256e+09,-1.465335e+09,-1.571165e+09,-1.095764e+09,-1.217244e+09,-1.129013e+09,-1.405244e+09,-1.737528e+09,-1.432091e+09
